In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/SemEval_26_Task8_MTRAG
!pwd
!ls

Mounted at /content/drive
/content/drive/MyDrive/SemEval_26_Task8_MTRAG
/content/drive/MyDrive/SemEval_26_Task8_MTRAG
 beir				    indexes		 'queries data'
 dataset			    indexes_qwen3_emb4b   README.md
 generator_reference		    logs		  src
 history_selected_rewrite_queries   paper_draft.pdf


In [ ]:
!pip -q install -U transformers sentence-transformers tqdm bitsandbytes accelerate
!pip install -U --no-cache-dir faiss-gpu-cu11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 196.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 405.5 MB/s eta 0:00:00


In [ ]:
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 123.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=253780426 sha256=4e2f9e39313266b1544b68138b15b91ee6221eccf14f7902b7c6620351340810
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn


In [ ]:
from sentence_transformers import SentenceTransformer
import torch

model = SentenceTransformer(
    "Qwen/Qwen3-Embedding-4B",
    device="cuda",
    model_kwargs={
        "attn_implementation": "flash_attention_2",
        "dtype": torch.float16,
    },
    tokenizer_kwargs={"padding_side": "left"},
)
model.max_seq_length = 512

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

In [ ]:
import json
from pathlib import Path

DOC_ID_KEYS = ["_id", "document_id", "id"]
TEXT_KEYS   = ["text", "contents", "body"]

def load_corpus_jsonl(path: Path):
    doc_ids, texts = [], []
    with path.open("r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)

            doc_id = next((obj[k] for k in DOC_ID_KEYS if k in obj), None)
            if doc_id is None:
                raise ValueError(f"Missing doc id in {obj.keys()}")

            text = next((obj[k] for k in TEXT_KEYS if k in obj and obj[k]), "")
            doc_ids.append(str(doc_id))
            texts.append(str(text))

    return doc_ids, texts


In [ ]:
from tqdm import tqdm
import numpy as np

def embed_documents(texts, batch_size=32):
    return model.encode(
        texts,
        batch_size=batch_size,
        show_progress_bar=True,
        convert_to_numpy=True,
        normalize_embeddings=True,
    ).astype("float32")


In [ ]:
import json
import numpy as np
import faiss
from pathlib import Path

def build_faiss_index(doc_emb: np.ndarray) -> faiss.Index:
    """
    doc_emb: shape (N_docs, dim), float32
    return: a FAISS index that can be searched with index.search(query_emb, k)
    """
    if doc_emb.dtype != np.float32:
        doc_emb = doc_emb.astype("float32")

    dim = doc_emb.shape[1]
    index = faiss.IndexFlatIP(dim)   # cosine if embeddings normalized
    index.add(doc_emb)               # add all document vectors into the index
    return index


def save_index_bundle(out_dir: Path, index: faiss.Index, doc_ids: list[str], doc_emb: np.ndarray):
    """
    Save everything needed for retrieval:
      - index.faiss  : the FAISS index
      - doc_ids.json : mapping faiss_id -> real document_id
      - doc_emb.npy  : cached document embeddings (optional but very useful)
    """
    out_dir.mkdir(parents=True, exist_ok=True)

    # 1) faiss index
    faiss.write_index(index, str(out_dir / "index.faiss"))

    # 2) doc id mapping
    with (out_dir / "doc_ids.json").open("w", encoding="utf-8") as f:
        json.dump(doc_ids, f, ensure_ascii=False)

    # 3) doc embeddings cache
    np.save(out_dir / "doc_emb.npy", doc_emb.astype("float32"))


def build_one_dataset_index(name: str, corpus_path: str | Path, out_root: Path, batch_size=32):
    """
    name: clapnq/govt/fiqa/cloud
    corpus_path: path to corpus.jsonl
    out_root: indexes_qwen3_emb4b/
    """
    corpus_path = Path(corpus_path)
    out_dir = out_root / f"{name}-qwen3-emb4b"

    print(f"\n=== Building index for {name} ===")
    doc_ids, texts = load_corpus_jsonl(corpus_path)
    print("docs:", len(doc_ids))

    doc_emb = embed_documents(texts, batch_size=batch_size)  # must be float32
    print("embeddings:", doc_emb.shape, doc_emb.dtype)

    index = build_faiss_index(doc_emb)
    save_index_bundle(out_dir, index, doc_ids, doc_emb)

    print("saved to:", out_dir)
    return out_dir



In [ ]:
from pathlib import Path

DATASETS = {
    "clapnq":   '/content/drive/MyDrive/SemEval_26_Task8_MTRAG/dataset/clapnq/corpus.jsonl',
}

OUT_DIR = Path("/content/drive/MyDrive/SemEval_26_Task8_MTRAG/indexes_qwen3_emb4b")
OUT_DIR.mkdir(parents=True, exist_ok=True)

for name, path in DATASETS.items():
    build_one_dataset_index(name, path, OUT_DIR, batch_size=32)


In [ ]:
from pathlib import Path

DATASETS = {
    "govt":     '/content/drive/MyDrive/SemEval_26_Task8_MTRAG/dataset/govt/corpus.jsonl',
    "fiqa":     '/content/drive/MyDrive/SemEval_26_Task8_MTRAG/dataset/fiqa/corpus.jsonl',
    "cloud":    '/content/drive/MyDrive/SemEval_26_Task8_MTRAG/dataset/cloud/corpus.jsonl',
}

OUT_DIR = Path("/content/drive/MyDrive/SemEval_26_Task8_MTRAG/indexes_qwen3_emb4b")
OUT_DIR.mkdir(parents=True, exist_ok=True)

for name, path in DATASETS.items():
    build_one_dataset_index(name, path, OUT_DIR, batch_size=4)